Attempting to load the occurrence dataframe and make it smaller by retaining just the necessary columns.

In [8]:
import os
import pandas as pd
import sys
from collections import Counter

from dwca.read import DwCAReader
from dwca.descriptors import shorten_term
from dwca.darwincore.utils import qualname as qn

In [2]:
home_dir = os.path.dirname(os.getcwd())
data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"

family_name_1 = "Sesiidae"
family_name_2 = "Drepanidae"

dwca_file_path_1     = os.path.join(data_dir,"dwca_files",family_name_1+".zip")
dwca_file_path_2     = os.path.join(data_dir,"dwca_files",family_name_2+".zip")
occurrence_file_path = os.path.join(data_dir,"dwca_files",family_name_2,"occurrence.txt")

In [5]:
fields_to_keep = [
    "id",
    "decimalLatitude",
    "decimalLongitude",
    "order",
    "family",
    "genus",
    "species",
    "acceptedScientificName",
    "year",
    "month",
    "day",
    "datasetName",
    "taxonID",
    "acceptedTaxonKey",
    "lifeStage",
    "basisOfRecord",
]

In [3]:
# Read the datafile descriptors for one
kwargs = {}
with DwCAReader(dwca_file_path_1) as dwca:
    # Get the file descriptor
    datafile_descriptor = dwca.get_descriptor_for("occurrence.txt")

In [13]:
# Just load the bigger file with usecols enabled
kwargs = {}
with DwCAReader(dwca_file_path_2) as dwca:
    
    kwargs['delimiter'] = datafile_descriptor.fields_terminated_by
    kwargs['skiprows'] = datafile_descriptor.lines_to_ignore
    kwargs['header'] = None
    kwargs['names'] = datafile_descriptor.short_headers
    kwargs['parse_dates'] = True
    kwargs['on_bad_lines'] = "skip"
    kwargs['usecols'] = fields_to_keep
    
    occ_df = pd.read_csv(occurrence_file_path, **kwargs)
    
    # Add a column for default values, if present in the file descriptor
    for field in datafile_descriptor.fields:
        field_default_value = field['default']
        if field_default_value is not None:
            occ_df[shorten_term(field['term'])] = field_default_value

/var/folders/bf/8z6pr__14lq801rwlk2hx9r40000gr/T/ipykernel_2837/2592414751.py:13: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  occ_df = pd.read_csv(occurrence_file_path, **kwargs)


In [14]:
# Save the big file
print(sys.getsizeof(occ_df)/1024/1024)

442.333270072937


In [20]:
# Read the big file in chunks and save only the smaller columns
kwargs = {}
with DwCAReader(dwca_file_path_2) as dwca:
    
    kwargs['delimiter'] = datafile_descriptor.fields_terminated_by
    kwargs['skiprows'] = datafile_descriptor.lines_to_ignore
    kwargs['header'] = None
    kwargs['names'] = datafile_descriptor.short_headers
    kwargs['parse_dates'] = True
    kwargs['on_bad_lines'] = "skip"
    kwargs['usecols'] = fields_to_keep
    kwargs['chunksize'] = 10000
    
    occ_df = []
    
    for chunk in pd.read_csv(occurrence_file_path, **kwargs):
            
        # Add a column for default values, if present in the file descriptor
        for field in datafile_descriptor.fields:
            field_default_value = field['default']
            if field_default_value is not None:
                chunk[shorten_term(field['term'])] = field_default_value
        
    
        occ_df.append(chunk)

occ_df_big = pd.concat(occ_df)
    

    

/var/folders/bf/8z6pr__14lq801rwlk2hx9r40000gr/T/ipykernel_2837/1601645251.py:16: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(occurrence_file_path, **kwargs):
/var/folders/bf/8z6pr__14lq801rwlk2hx9r40000gr/T/ipykernel_2837/1601645251.py:16: DtypeWarning: Columns (61) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(occurrence_file_path, **kwargs):
/var/folders/bf/8z6pr__14lq801rwlk2hx9r40000gr/T/ipykernel_2837/1601645251.py:16: DtypeWarning: Columns (61,180) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(occurrence_file_path, **kwargs):
/var/folders/bf/8z6pr__14lq801rwlk2hx9r40000gr/T/ipykernel_2837/1601645251.py:16: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(occurrence_file_path, **kwargs):
/var/folders/bf/8

In [22]:
occ_df_big.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 660000 to 660049
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      50 non-null     int64  
 1   datasetName             0 non-null      float64
 2   basisOfRecord           50 non-null     object 
 3   lifeStage               0 non-null      float64
 4   year                    50 non-null     int64  
 5   month                   50 non-null     int64  
 6   day                     50 non-null     int64  
 7   decimalLatitude         50 non-null     float64
 8   decimalLongitude        50 non-null     float64
 9   taxonID                 2 non-null      float64
 10  order                   50 non-null     object 
 11  family                  50 non-null     object 
 12  genus                   50 non-null     object 
 13  acceptedTaxonKey        50 non-null     int64  
 14  species                 50 non-null